在本章中，我们将学到如下内容：
+ 介绍常用分类算法的概念
+ 使用scikit-learn机器学习库
+ 选择机器学习算法时需要注意的问题  

5  
# 3.1 分类算法的选择
再次借用一下“没有免费午餐”理论：没有任何一种分类器可以在所有可能的应用场景下都有良好的表现  
总而言之，分类器的性能、计算能力和预测能力，在很大程度上都依赖于用于模型训练的相关数据。训练机器学习算法所涉及的五个主要步骤可以概述如下：  
1. 特征的选择  
2. 确定性能评价标准
3. 选择分类器及其优化算法
4. 对模型性能的评估
5. 算法的调优
5  

# 3.2 初涉scikit-learn的使用
## 使用sckit-learn训练感知器

In [ ]:
'''
python	sklearn	datasets	load_iris() feature_names
python	sklearn	datasets	load_iris() target
python	sklearn	datasets	load_iris() data
'''
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
iris.feature_names

为了评估训练得到的模型在未知数据上的表现，我们进一步将数据集划分为训练数据集和测试数据集 

In [ ]:
'''
python	sklearn	model_selection	train_test_split()
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 0)

在此，我们将使用scikit-learn的preprocessing模块中的standardscaler类对特征进行标准化处理  

In [ ]:
'''
5
'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

需要注意的是，我们要使用相同的缩放参数分别处理训练和测试数据集，以保证它们的值是彼此相当的  
在对训练数据做了标准化处理后，我们现在可以训练感知器模型了。scikit-learn中的大多数算法本身就使用一对多方法来支持多类别分类，因此，我们可以将三种鸢尾花的数据同时输入到感知器中    

In [ ]:
'''
python	sklearn	Linear Models	Perceptron() max_iter
python	sklearn	Linear Models	Perceptron() eta0
'''
from sklearn.linear_model import Perceptron
ppn = Perceptron(max_iter = 40, eta0 = 0.1, random_state = 0)
ppn.fit(X_train_std, y_train)

此模型的参数eta0与我们自行实现的感知器中的学习速率eta等价，而参数max_iter定义了迭代的次数  
与第2章中实现的感知器一样，使用scikit-learn完成模型的训练后，就可以在测试数据集上使用predict方法进行预测了 5 

In [ ]:
'''
python	str	format()	format() d
'''
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: {:d}'.format((y_test != y_pred).sum()))

许多机器学习从业者通常使用准确率而不是误分类来评判一个模型，它们关系如下：  
1 - 误分类率 = 准确率  
可以通过metrics模块计算感知器在测试数据集上的分类准确率：  

In [ ]:
'''
python	sklearn	Classification metrics	accuracy_score()
'''
from sklearn.metrics import accuracy_score
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

绘制刚刚训练过的模型的决策区域，我们做少许修改：使用小圆圈来高亮显示来自测试数据集的样本 5  

In [ ]:
'''
python	numpy	Array manipulation routines	unique()
python	matplotlib	colors	ListedColormap()
python	numpy	The N-dimensional array (ndarray)	min()
python	numpy	The N-dimensional array (ndarray)	max()
python	numpy	Array creation routines	meshgrid()
python	matplotlib	Pyplot function overview	contourf()
python	matplotlib	Pyplot function overview	scatter() alpha
python	matplotlib	collections	label
'''

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier,
                          test_idx = None, resolution = 0.02):
    # 设置散点生成器和色图
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # 绘制决策边界
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                          np.arange(x2_min, x2_max, resolution)) 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha = 0.4, cmap = cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # 绘制样本分类
    X_test, y_test = X[test_idx, :], y[test_idx]
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
                   alpha = 0.8, color = cmap(idx),
                   marker = markers[idx], label = cl)

    # 高亮测试样本
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], c = 'yellow',
                   alpha = 0.5, linewidth = 1, marker = 'o',
                   s = 55, label = 'test set')

In [ ]:
'''
python	numpy	Array manipulation routines	vstack()
python	numpy	Array manipulation routines	hstack()
python	matplotlib	Pyplot function overview	legend() loc

'''
import numpy as np
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X = X_combined_std,
                     y = y_combined,
                     classifier = ppn,
                     test_idx = range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

从结果呈现的图像中我们可以看到，无法通过一个线性决策边界完美区分三类样本  
对于无法完美线性可分的数据集，感知器算法将永远无法收敛，这也是在实践中一般不推荐使用感知器算法的原因 5
# 3.3 逻辑斯蒂回归中的类别概率
